In [1]:
from models.q_network import *
from game.game_utils import Card, cards_to_array, Deck, actions_to_array, Action
from game.utils import variable

In [2]:
n_actions = 14
hdim = 10
f = CardFeaturizer(hdim)
q = QNetwork(n_actions, hdim, f)
pi = PiNetwork(n_actions, hdim, f, q_network=q)

# How it should be used

Get some cards

In [12]:
deck = Deck()
deck.populate()
deck.shuffle()
hand = variable(cards_to_array(deck.cards[:2])).resize(1,13,4)
board = variable(cards_to_array(deck.cards[2:5])).resize(1,3,13,4)

Create a game (preflop passed and now it is flop)

In [13]:
actions_dict = {b_round: {player: [] for player in range(2)} for b_round in range(4)}
actions_dict[0][0].append(Action('call', 1))
actions_dict[0][1].append(Action('check', 0))
preflop_plays, flop_plays, turn_plays, river_plays = actions_to_array(actions_dict)
preflop_plays = variable(preflop_plays).resize(1,6,5,2)
flop_plays = variable(flop_plays).resize(1,6,5,2)
turn_plays = variable(turn_plays).resize(1,6,5,2)
river_plays = variable(river_plays).resize(1,6,5,2)

Other variables

In [14]:
pot = variable(10).resize(1,1)
stack = variable(98).resize(1,1)
opponent_stack = variable(98).resize(1,1)
BB = variable(2).resize(1,1)
dealer = variable(0).resize(1,1)


Test q and pi

In [15]:
q.forward(hand, board,  pot, stack, opponent_stack, BB, dealer, preflop_plays, flop_plays, turn_plays, river_plays)

Variable containing:

Columns 0 to 9 
-0.0867 -0.5641 -0.8212 -1.1275 -0.2186  0.5801  0.5392 -0.0816  0.7848 -0.5914

Columns 10 to 13 
-0.8976  0.0751 -0.8128  0.6367
[torch.FloatTensor of size 1x14]

In [24]:
pi.forward(card_feat, flop_feat, turn_feat, river_feat,  pot, stack, opponent_stack, BB, dealer, preflop_plays, flop_plays, turn_plays, river_plays)

Variable containing:

Columns 0 to 9 
 0.0204  0.0408  0.0282  0.1921  0.0303  0.0153  0.0312  0.0741  0.2195  0.0626

Columns 10 to 13 
 0.0776  0.0703  0.0724  0.0652
[torch.FloatTensor of size 1x14]